In [5]:
import numpy as np
import rasterio
from rasterio.enums import Resampling

# 打开地理数据和冰川数据
geo_data = rasterio.open('D:\CSSP\SJY/geo_dem_for11.tif')
glacier_data = rasterio.open('D:\CSSP\SJY/30glcsjy.tif')

# 重采样冰川数据到地理数据的分辨率
glacier_resampled = glacier_data.read(
    out_shape=(geo_data.height, geo_data.width),
    resampling=Resampling.bilinear
)

# 计算每个单元的冰川面积
glacier_area = glacier_resampled * (geo_data.res[0] * geo_data.res[1])

# 计算冰川占比
glacier_percentage = glacier_area / (geo_data.res[0] * geo_data.res[1])

# 保存结果
with rasterio.open('glacier_percentage.tif', 'w', **geo_data.meta) as dst:
    dst.write(glacier_percentage, 1)


ValueError: Source shape (1, 1, 139, 289) is inconsistent with given indexes 1

In [8]:
import numpy as np
from osgeo import gdal
import xarray as xr
from tqdm import tqdm  # 导入 tqdm 用于进度条

# 读取输入的高分辨率冰川栅格（0.0027度）
input_glacier_raster = r'D:\Data\Glacier_Global\RGI2000-v7.0-G-global\RGI2000v7_global100m.tif'

# 打开冰川栅格文件
glacier_ds = gdal.Open(input_glacier_raster)
if glacier_ds is None:
    raise FileNotFoundError(f"Failed to open {input_glacier_raster}")

glacier_band = glacier_ds.GetRasterBand(1)

# 获取栅格文件的大小
xsize = glacier_band.XSize
ysize = glacier_band.YSize

# 初始化一个空的 grid_data 数组
grid_data = np.zeros((560, 1440), dtype=float)

# 获取冰川数据的仿射变换参数
glacier_geotransform = glacier_ds.GetGeoTransform()

# 计算冰川栅格的分辨率和大小
glacier_x_res = glacier_geotransform[1]  # 0.008333度
glacier_y_res = abs(glacier_geotransform[5])  # 0.008333度
grid_x_res = 0.25  # 0.25度
grid_y_res = 0.25  # 0.25度

# 计算每个粗分辨率网格中有多少个高分辨率像元
x_ratio = int(grid_x_res / glacier_x_res)
y_ratio = int(grid_y_res / glacier_y_res)

print(f"x_ratio: {x_ratio}, y_ratio: {y_ratio}")

# 初始化输出数组，用于存储每个粗分辨率网格的冰川覆盖率
output_coverage = np.zeros_like(grid_data, dtype=float)

# 定义分块大小（可以根据内存大小调整）
block_size_x = 1000
block_size_y = 1000

# 遍历每个粗分辨率网格，并添加 tqdm 进度条
for i in tqdm(range(0, ysize, block_size_y), desc="Processing Rows", unit="row"):
    for j in range(0, xsize, block_size_x):
        # 计算块大小
        x_read_size = min(block_size_x, xsize - j)
        y_read_size = min(block_size_y, ysize - i)
        
        # 读取数据块
        glacier_subset = glacier_band.ReadAsArray(j, i, x_read_size, y_read_size)
        
        if glacier_subset is None:
            continue
        
        # 在这里处理分块数据，计算每个块的冰川覆盖率
        # 遍历该块中每个像素，计算覆盖率
        for m in range(0, y_read_size, y_ratio):
            for n in range(0, x_read_size, x_ratio):
                x_start = j + n
                y_start = i + m
                x_end = x_start + x_ratio
                y_end = y_start + y_ratio
                
                # 提取高分辨率冰川数据子集
                glacier_block = glacier_subset[m:m + y_ratio, n:n + x_ratio]
                
                # 检查子集是否为空
                if glacier_block.size == 0:
                    continue
                
                # 计算冰川覆盖的像元数量（假设冰川覆盖像元的值为1，非冰川为0）
                glacier_covered = np.sum(glacier_block > 0)
                total_cells = glacier_block.size
                
                # 计算覆盖率
                coverage_ratio = glacier_covered / total_cells
                
                # 存储在输出数组中
                output_coverage[m // y_ratio, n // x_ratio] = coverage_ratio

# 输出结果：每个粗分辨率网格中的冰川覆盖率
print("Grid Glacier Coverage Ratios:")

# 获取网格的经纬度信息
lat_size, lon_size = grid_data.shape
lat_start = 85
lon_start = -180

latitudes = np.linspace(lat_start, lat_start - lat_size * grid_y_res, lat_size)
longitudes = np.linspace(lon_start, lon_start + lon_size * grid_x_res, lon_size)

# 创建 xarray 数据集
ds = xr.Dataset(
    {
        "glacier_coverage": (["lat", "lon"], output_coverage)
    },
    coords={
        "lat": latitudes,
        "lon": longitudes
    }
)

# 保存为 NetCDF 文件
output_nc_file = r'D:\Data\Glacier_Global\glacier_coverage_global.nc'
ds.to_netcdf(output_nc_file)

print(f"Glacier coverage saved to {output_nc_file}")


x_ratio: 300, y_ratio: 300


Processing Rows:   1%|          | 2/195 [00:06<10:45,  3.34s/row]


KeyboardInterrupt: 

In [ ]:
import numpy as np
from osgeo import gdal
import xarray as xr

# 读取输入的高分辨率冰川栅格（0.0027度）
input_glacier_raster = r'D:\CSSP\SJY/1976TPG_SJY.tif'
# 读取输入的地理栅格（0.05度）
input_grid_raster = r'D:\CSSP\SJY\INT_geo_sjy.tif'

# 打开冰川栅格文件
glacier_ds = gdal.Open(input_glacier_raster)
glacier_band = glacier_ds.GetRasterBand(1)
glacier_data = glacier_band.ReadAsArray()

# 打开网格栅格文件
grid_ds = gdal.Open(input_grid_raster)
grid_band = grid_ds.GetRasterBand(1)
grid_data = grid_band.ReadAsArray()
x_ratio , y_ratio = int(42934/289),int(28790/139)
full_output_coverage = np.zeros(grid_data.shape,  dtype=float)
y_start , x_start =0,0
grid_y_start,grid_x_start = 0, 0
# 遍历每个粗分辨率网格
for i in range(139):
    for j in range(289):
        # 计算高分辨率冰川数据中对应的像元索引
        y_start =  (i - grid_y_start) * y_ratio
        x_start = (j - grid_x_start) * x_ratio
        y_end = y_start + y_ratio
        x_end = x_start + x_ratio

        # 提取高分辨率冰川数据子集
        glacier_subset = glacier_data[y_start:y_end, x_start:x_end]
        print(glacier_subset)
        # 计算冰川覆盖的像元数量（假设冰川覆盖像元的值9为 1）
        glacier_covered = np.sum((glacier_subset < 65534) & (glacier_subset > 0))

        total_cells = glacier_subset.size
        if glacier_covered >0:
            print(y_start , y_end , x_start , x_end,glacier_covered,1)
        # 计算覆盖率
        coverage_ratio = glacier_covered / total_cells if total_cells > 0 else 0

        # 存储在完整输出数组中
        full_output_coverage[i, j] = coverage_ratio

# 获取完整网格的经纬度信息
latitudes = np.linspace(103.115, 88.665, grid_data.shape[0])
longitudes = np.linspace(30.635, 37.585, grid_data.shape[1])

# 创建 xarray 数据集
ds = xr.Dataset(
    {
        "glacier_coverage": (["lat", "lon"], full_output_coverage[::-1, :]),
    },
    coords={
        "lat": latitudes,
        "lon": longitudes
    }
)

# 保存为 NetCDF 文件
output_nc_file = r"D:\CSSP\SJY\SJY1976glacier_coverage_output1.nc"
ds.to_netcdf(output_nc_file)

print(f"冰川覆盖率已保存到 {output_nc_file}")


In [12]:

# 读取输入的高分辨率冰川栅格（0.0027度）
input_glacier_raster = r'D:\CSSP\SJY/1976TPG_SJYbin.tif'
# 读取输入的地理栅格（0.05度）
input_grid_raster = r'D:\CSSP\SJY\fildem.tif'

# 打开冰川栅格文件
glacier_ds = gdal.Open(input_glacier_raster)
glacier_band = glacier_ds.GetRasterBand(1)
glacier_data = glacier_band.ReadAsArray()

# 打开网格栅格文件
grid_ds = gdal.Open(input_grid_raster)
grid_band = grid_ds.GetRasterBand(1)
grid_data = grid_band.ReadAsArray()

# 获取仿射变换参数
glacier_geotransform = glacier_ds.GetGeoTransform()
grid_geotransform = grid_ds.GetGeoTransform()

# 获取冰川和地理数据的范围
glacier_xmin = grid_geotransform[0]
glacier_ymax = grid_geotransform[3]
glacier_xmax = glacier_xmin + grid_geotransform[1] * grid_data.shape[1]
glacier_ymin =glacier_ymax + grid_geotransform[5] * grid_data.shape[0]
print(glacier_xmin,glacier_ymax,glacier_xmax,glacier_ymin)
grid_xmin = grid_geotransform[0]
grid_ymax = grid_geotransform[3]
grid_xmax = grid_xmin + grid_geotransform[1] * grid_data.shape[1]
grid_ymin = grid_ymax + grid_geotransform[5] * grid_data.shape[0]

# 计算重叠区域
overlap_xmin = max(glacier_xmin, grid_xmin)
overlap_ymax = min(glacier_ymax, grid_ymax)
overlap_xmax = min(glacier_xmax, grid_xmax)
overlap_ymin = max(glacier_ymin, grid_ymin)

# 如果没有重叠区域，退出
if overlap_xmin >= overlap_xmax or overlap_ymin >= overlap_ymax:
    print("地理数据和冰川数据之间没有重叠区域。")
    exit()

# 计算重叠区域的行列索引
glacier_x_start = int((overlap_xmin - glacier_xmin) / 0.00027)
glacier_y_start = int((glacier_ymax - overlap_ymax) / 0.00027)
glacier_x_end = int((overlap_xmax - glacier_xmin) /  0.00027)
glacier_y_end = int((glacier_ymax - overlap_ymin) /  0.00027)

grid_x_start = int((overlap_xmin - grid_xmin) / grid_geotransform[1])
grid_y_start = int((grid_ymax - overlap_ymax) / abs(grid_geotransform[5]))
grid_x_end = int((overlap_xmax - grid_xmin) / grid_geotransform[1])
grid_y_end = int((grid_ymax - overlap_ymin) / abs(grid_geotransform[5]))
print(grid_geotransform)
# 计算每个粗分辨率网格中包含的高分辨率像元数量
glacier_x_res = 0.00027
glacier_y_res = 0.00027
grid_x_res = grid_geotransform[1]
grid_y_res = abs(grid_geotransform[5])
x_ratio = int(grid_x_res / glacier_x_res)
y_ratio = int(grid_y_res / glacier_y_res)





-0.5 0.5 288.5 -138.5
(-0.5, 1.0, 0.0, 0.5, 0.0, -1.0)


In [27]:
import numpy as np
from osgeo import gdal
import xarray as xr

# 读取输入的高分辨率冰川栅格（0.0027度）
input_glacier_raster = r'D:/CSSP/SJY/3km/1976TPG_SJYbin_tolam3km.tif'
# 读取输入的地理栅格（0.05度）
input_grid_raster = r'D:/CSSP/SJY/3km//dem_for_arc11.tif'

# 打开冰川栅格文件
glacier_ds = gdal.Open(input_glacier_raster)
glacier_band = glacier_ds.GetRasterBand(1)
glacier_data = glacier_band.ReadAsArray()
print(glacier_data.shape)
# 打开网格栅格文件
grid_ds = gdal.Open(input_grid_raster)
grid_band = grid_ds.GetRasterBand(1)
grid_data = grid_band.ReadAsArray()


# 计算网格与冰川数据的比例
x_ratio, y_ratio = int(glacier_data.shape[0] / grid_data.shape[0]), int(glacier_data.shape[1]/ grid_data.shape[1])

# 初始化输出数组
full_output_coverage = np.zeros(grid_data.shape, dtype=float)
# 获取网格栅格的地理信息
geo_transform = grid_ds.GetGeoTransform()

# 提取经纬度范围
grid_xmin = geo_transform[0]
grid_ymax = geo_transform[3]
grid_xmax = geo_transform[0] + geo_transform[1] * grid_data.shape[1]
grid_ymin = geo_transform[3] + geo_transform[5] * grid_data.shape[0]
print(grid_xmin, grid_ymax, grid_xmax, grid_ymin)
# geo_transform[0] = 左上角的X坐标（经度）
# geo_transform[3] = 左上角的Y坐标（纬度）
# geo_transform[1] = 每个像素的宽度
# geo_transform[5] = 每个像素的高度（负值表示从上到下）

# 获取完整网格的经纬度信息
latitudes = np.arange(0, grid_data.shape[0])
longitudes = np.arange(0, grid_data.shape[1])

# 计算高分辨率栅格的对应区域偏移（优化）
grid_y_start = 0  # Assuming grid_y_start is 0, adjust if needed
grid_x_start = 0  # Assuming grid_x_start is 0, adjust if needed

# 遍历网格数据的每个单元
for i in range(grid_data.shape[0]):  # 遍历纬度（行）
    for j in range(grid_data.shape[1]):  # 遍历经度（列）
        # 计算高分辨率冰川数据对应的区域
        y_start = (i - grid_y_start) * y_ratio
        x_start = (j - grid_x_start) * x_ratio
        y_end = y_start + y_ratio
        x_end = x_start + x_ratio

        # 提取高分辨率冰川数据子集
        glacier_subset = glacier_data[y_start:y_end, x_start:x_end]

        # 计算冰川覆盖率
        glacier_covered = np.sum(glacier_subset == 1)  # 假设冰川覆盖像元值为 1
        total_cells = glacier_subset.size
        coverage_ratio = glacier_covered / total_cells if total_cells > 0 else 0
        
        # 存储到完整输出数组中
        full_output_coverage[i, j] = coverage_ratio
# 右移7个像素（列方向）的纯数据操作
shift_pixels = 7 # 可修改为任意移动量
shifted_data = np.roll(full_output_coverage, shift_pixels, axis=1)  # axis=1表示列方向
# 创建 xarray 数据集
ds = xr.Dataset(
    {
        "glacier_coverage": (["lat", "lon"], shifted_data [::-1, :]),  # 反转纬度顺序
    },
    coords={
        "lat": latitudes,
        "lon": longitudes
    }
)

# 保存为 NetCDF 文件
output_nc_file = r"D:\CSSP\SJY\1976glacier_coverage_output3km.nc"
ds.to_netcdf(output_nc_file)

print(f"冰川覆盖率已保存到 {output_nc_file}")



(27195, 46973)
-713135.5 4674367.7 726864.5 3894367.7
冰川覆盖率已保存到 D:\CSSP\SJY\1976glacier_coverage_output3km.nc


In [16]:
print(gdal.VersionInfo())  # 输出版本信息（需 >= 3.0）
print(gdal.GetDriverByName('GTiff').GetMetadata())  # 检查 TIFF 驱动是否可用


3060200
{'DCAP_COORDINATE_EPOCH': 'YES', 'DCAP_CREATE': 'YES', 'DCAP_CREATECOPY': 'YES', 'DCAP_OPEN': 'YES', 'DCAP_RASTER': 'YES', 'DCAP_VIRTUALIO': 'YES', 'DMD_CREATIONDATATYPES': 'Byte UInt16 Int16 UInt32 Int32 Float32 Float64 CInt16 CInt32 CFloat32 CFloat64', 'DMD_CREATIONOPTIONLIST': "<CreationOptionList>   <Option name='COMPRESS' type='string-select'>       <Value>NONE</Value>       <Value>LZW</Value>       <Value>PACKBITS</Value>       <Value>JPEG</Value>       <Value>CCITTRLE</Value>       <Value>CCITTFAX3</Value>       <Value>CCITTFAX4</Value>       <Value>DEFLATE</Value>       <Value>LZMA</Value>       <Value>ZSTD</Value>       <Value>LERC</Value>       <Value>LERC_DEFLATE</Value>       <Value>LERC_ZSTD</Value>   </Option>   <Option name='PREDICTOR' type='int' description='Predictor Type (1=default, 2=horizontal differencing, 3=floating point prediction)'/>   <Option name='DISCARD_LSB' type='string' description='Number of least-significant bits to set to clear as a single valu

In [ ]:
import numpy as np
from osgeo import gdal
import xarray as xr

# 读取输入的高分辨率冰川栅格（0.0027度）
input_glacier_raster = r'D:\CSSP\SJY/1976TPG_SJYbin.tif'
# 读取输入的地理栅格（0.05度）
input_grid_raster = r'D:\CSSP\SJY\dem_for_arc1.tif'

# 打开冰川栅格文件
glacier_ds = gdal.Open(input_glacier_raster)
glacier_band = glacier_ds.GetRasterBand(1)
glacier_data = glacier_band.ReadAsArray()

# 打开网格栅格文件
grid_ds = gdal.Open(input_grid_raster)
grid_band = grid_ds.GetRasterBand(1)
grid_data = grid_band.ReadAsArray()
print(grid_data.shape)
# 比例因子计算
x_ratio, y_ratio = int(43934/289), int(28790/139)

# 完整输出数组
full_output_coverage = np.zeros(grid_data.shape, dtype=float)

# 定义网格的起始值（根据实际情况调整）
grid_x_start, grid_y_start = 0, 1

grid_xmin = geo_transform[0]
grid_ymax = geo_transform[3]
grid_xmax = geo_transform[0] + geo_transform[1] * grid_data.shape[1]
grid_ymin = geo_transform[3] + geo_transform[5] * grid_data.shape[0]

# 遍历每个粗分辨率网格
for i in range(139):
    for j in range(289):
        # 计算高分辨率冰川数据中对应的像元索引
        y_start = (i - grid_y_start) * y_ratio
        x_start = (j - grid_x_start) * x_ratio
        y_end = y_start + y_ratio
        x_end = x_start + x_ratio
        # 提取高分辨率冰川数据子集
        glacier_subset = glacier_data[y_start:y_end, x_start:x_end]

        # 计算冰川覆盖的像元数量（假设冰川覆盖像元的值9为 1）
        glacier_covered = np.sum(glacier_subset == 1)
        total_cells = glacier_subset.size

        # 计算覆盖率
        coverage_ratio = glacier_covered / total_cells if total_cells > 0 else 0

        # 存储在完整输出数组中
        full_output_coverage[i, j] = coverage_ratio

# 获取完整网格的经纬度信息
latitudes = np.linspace(grid_ymax, grid_ymin, grid_data.shape[0])
longitudes = np.linspace(grid_xmin, grid_xmax, grid_data.shape[1])

# 创建 xarray 数据集
ds = xr.Dataset(
    {
        "glacier_coverage": (["lat", "lon"], full_output_coverage[::-1, :]),
    },
    coords={
        "lat": latitudes,
        "lon": longitudes
    }
)

# 保存为 NetCDF 文件
output_nc_file = r"D:\CSSP\SJY\1976SJYglacier_coverage_output1.nc"
ds.to_netcdf(output_nc_file, encoding={'glacier_coverage': {'zlib': True, 'complevel': 5}})

print(f"冰川覆盖率已保存到 {output_nc_file}")


(139, 289)
冰川覆盖率已保存到 D:\CSSP\SJY\1976SJYglacier_coverage_output1.nc


In [ ]:
import numpy as np
from osgeo import gdal
import xarray as xr

def check_file_exists(file_path):
    """
    检查文件是否存在。
    """
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"文件 {file_path} 不存在")

def get_geospatial_info(raster_file):
    """
    从栅格文件中读取地理信息，如经纬度范围、分辨率、行列数等。
    """
    # 打开栅格文件
    ds = gdal.Open(raster_file)
    
    # 获取地理变换信息 (GeoTransform)
    geotransform = ds.GetGeoTransform()
    # 获取栅格的空间分辨率
    pixel_size_x = geotransform[1]  # 每个像素的经度跨度
    pixel_size_y = geotransform[5]  # 每个像素的纬度跨度
    
    # 获取经纬度范围（左上角和右下角坐标）
    min_lon = geotransform[0]
    max_lat = geotransform[3]
    max_lon = min_lon + pixel_size_x * ds.RasterXSize
    min_lat = max_lat + pixel_size_y * ds.RasterYSize
    
    # 获取栅格的行列数
    rows = ds.RasterYSize
    cols = ds.RasterXSize
    
    return min_lat, max_lat, min_lon, max_lon, rows, cols, pixel_size_y, pixel_size_x

def calculate_glacier_coverage(input_glacier_raster, input_grid_raster, output_file, file_format='netcdf', glacier_value_range=(6670, 12000)):
    """
    计算并保存冰川覆盖率到文件。

    参数:
    - input_glacier_raster: 高分辨率冰川栅格文件路径
    - input_grid_raster: 网格栅格文件路径
    - output_file: 输出的文件路径
    - file_format: 输出文件格式，支持 'netcdf' 和 'geotiff'
    - glacier_value_range: 用于确定冰川覆盖的值范围 (min, max)
    """
    
    # 检查文件路径
    check_file_exists(input_glacier_raster)
    check_file_exists(input_grid_raster)

    # 获取冰川栅格文件的地理信息
    glacier_min_lat, glacier_max_lat, glacier_min_lon, glacier_max_lon, glacier_rows, glacier_cols, glacier_pixel_size_y, glacier_pixel_size_x = get_geospatial_info(input_glacier_raster)
    print(get_geospatial_info(input_glacier_raster))
    # 获取网格栅格文件的地理信息
    grid_min_lat, grid_max_lat, grid_min_lon, grid_max_lon, grid_rows, grid_cols, grid_pixel_size_y, grid_pixel_size_x = get_geospatial_info(input_grid_raster)
    print( get_geospatial_info(input_grid_raster))
    # 设置经纬度范围（使用地理栅格的范围）
    lat_range = (grid_max_lat, grid_min_lat)
    lon_range = (grid_min_lon, grid_max_lon)
    
    # 使用栅格的行列数和分辨率计算分辨率比例
    y_ratio = glacier_rows / grid_rows
    x_ratio = glacier_cols / grid_cols
    # 打开冰川栅格文件
    glacier_ds = gdal.Open(input_glacier_raster)
    glacier_band = glacier_ds.GetRasterBand(1)
    glacier_data = glacier_band.ReadAsArray()

    # 打开网格栅格文件
    grid_ds = gdal.Open(input_grid_raster)
    grid_band = grid_ds.GetRasterBand(1)
    grid_data = grid_band.ReadAsArray()

    # 初始化输出覆盖率数组
    full_output_coverage = np.zeros(grid_data.shape, dtype=float)
    # 获取经纬度信息
    latitudes = np.linspace(lat_range[0], lat_range[1], grid_data.shape[0])
    longitudes = np.linspace(lon_range[0], lon_range[1], grid_data.shape[1])
    # 遍历每个粗分辨率网格
    for i in range(grid_rows):
        for j in range(grid_cols):
            # 计算高分辨率冰川数据中对应的像元索引
            y_start = i * y_ratio
            x_start = j * x_ratio
            y_end = y_start + y_ratio
            x_end = x_start + x_ratio

            # 提取高分辨率冰川数据子集
            glacier_subset = glacier_data[int(y_start):int(y_end), int(x_start):int(x_end)]

            # 计算冰川覆盖的像元数量（假设冰川覆盖像元的值在指定范围内）
            glacier_covered = np.sum((glacier_subset >= glacier_value_range[0]) & (glacier_subset <= glacier_value_range[1]))

            total_cells = glacier_subset.size
            # 计算覆盖率
            coverage_ratio = glacier_covered / total_cells if total_cells > 0 else 0

            # 存储在完整输出数组中
            full_output_coverage[i, j] = coverage_ratio

    # 根据指定的输出格式保存数据
    if file_format.lower() == 'netcdf':
        # 创建 xarray 数据集
        ds = xr.Dataset(
            {
                "glacier_coverage": (["lat", "lon"], full_output_coverage[::-1, :]),  # 纬度反转
            },
            coords={
                "lat": latitudes,
                "lon": longitudes
            }
        )
        # 保存为 NetCDF 文件
        ds.to_netcdf(output_file)
        print(f"冰川覆盖率已保存到 {output_file}")
    
    elif file_format.lower() == 'geotiff':
        # 创建输出 GeoTIFF 文件
        driver = gdal.GetDriverByName('GTiff')
        out_ds = driver.Create(output_file, grid_data.shape[1], grid_data.shape[0], 1, gdal.GDT_Float32)
        out_ds.SetGeoTransform(grid_ds.GetGeoTransform())  # 保留原始栅格的空间参考
        out_ds.SetProjection(grid_ds.GetProjection())
        out_band = out_ds.GetRasterBand(1)
        out_band.WriteArray(full_output_coverage[::-1, :])  # 纬度反转
        out_band.FlushCache()
        print(f"冰川覆盖率已保存为 GeoTIFF 格式: {output_file}")
    
    else:
        raise ValueError(f"不支持的输出格式: {file_format}")

# 调用函数
input_glacier_raster = r'D:\CSSP\tuotuoriver/tuotuoriverglc2.tif'
input_grid_raster = r'D:\CSSP\tuotuoriver\tuotuorivermask1.tif'
output_file = r"D:\CSSP\tuotuoriver\TTH1976glacier_coverage_output1.nc"  # 可以更改为 '.tif' 格式
calculate_glacier_coverage(input_glacier_raster, input_grid_raster, output_file, file_format='netcdf')



(407188.52081747353, 597598.5208174735, -382700.06441778707, -211040.06441778707, 6347, 5722, -30.0, 30.0)
(32.80833333345316, 35.816666666786325, 88.5583333333332, 93.44999999999959, 361, 587, -0.00833333333333286, 0.008333333333332872)
冰川覆盖率已保存到 D:\CSSP\tuotuoriver\TTH1976glacier_coverage_output1.nc
